---
layout: post
title: "The Case Against Ranked-Choice Voting: Part II"
tags: [Elections, Politics]
--- 

This is part II of my post laying out the case against ranked-choice voting. In this post, I take the theoretical results from the previous post and see if I can find instances of them in a real ranked-choice election. I'm going to use the 2009 election for mayor of Burlington, Vermont as my test case. I chose this because the votes have been carefully counted and uploaded by Juho Laatu and Warren D. Smith, and they have [made them available online](https://rangevoting.org/JLburl09.txt). As I was researching this, I came across the site https://rangevoting.org/ where I learned a lot about voting systems. Some of the examples below I first found out about on their site.

> Note: There are minor disagreements in how certain invalid ballots were counted, so the results here do not exactly match all the tabulations online. However, they all seem to be within 5 votes of each other and the discrepancies don't affect any results.

In this election there were five candidates. Here are the candidates and their parties:
* Bob Kiss - Progressive
* Andy Montroll - Democrat
* Kurt Wright - Republican
* Dan Smith - Independent
* James Simpson - Green

I'm going to start by loading the election results and simulating the election to ensure that I get the same result as the true election.

In [1]:
import re
from votesim import *

In [2]:
file = 'JLburl09.txt'

In [3]:
with open(file, 'r') as f:
    a = f.readlines()

In [4]:
regex = '(\d+): ((\w>?)+)'

In [5]:
kiss = Candidate("Bob Kiss")
wright = Candidate("Kurt Wright")
montroll = Candidate("Andy Montroll")
smith = Candidate("Dan Smith")
simpson = Candidate("James Simpson")
write_in = Candidate("Write-in")

candidates = [kiss, wright, montroll, smith, simpson, write_in]

In [6]:
cand_map = {'K': kiss,
           'M': montroll,
           'N': simpson,
           'H': smith,
           'W': wright,
           'R': write_in}

In [7]:
all_ballots = []
for i in range(18, 395):
    m = re.search(regex, a[i])
    num_ballots = int(m.group(1))
    ranked_cands = m.group(2).split('>')
    ranked_can_list = []
    for c in ranked_cands:
        ranked_can_list.append(cand_map[c])
    my_bal = Ballot(ranked_can_list)
    all_ballots.extend([my_bal] * num_ballots)

In [8]:
election_result = instant_runoff_voting(candidates, all_ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Kurt Wright       2949  Active
Bob Kiss          2585  Active
Andy Montroll     2062  Active
Dan Smith         1306  Rejected
Write-in            36  Rejected
James Simpson       35  Rejected

ROUND 2
Candidate        Votes  Status
-------------  -------  --------
Kurt Wright       3292  Active
Bob Kiss          2981  Active
Andy Montroll     2553  Rejected
Dan Smith            0  Rejected
Write-in             0  Rejected
James Simpson        0  Rejected
Blank votes        147  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Bob Kiss          4313  Elected
Kurt Wright       4058  Rejected
Andy Montroll        0  Rejected
Dan Smith            0  Rejected
Write-in             0  Rejected
James Simpson        0  Rejected
Blank votes        602  Rejected



Bob Kiss wins the election, which is what happened in the real election. Not let's go through the different failure modes of ranked-choice voting and see if we can find them here.

## Ranked-Choice Voting Encourages Strategic Voting

There is clear evidence that strategic voting would have been beneficial in this election. Lots of voters had the preference of Wright, then Montroll, then Kiss. With the *promise* of ranked-choice voting, they should be able to express those preferences. According to the promise, if they vote for Wright and then Montroll, that can't hurt Montroll's chances against Kiss, because Montroll is ranked first and Wright was ranked next. But let's look at what really happened.

Let's look at the election from the perspective of voter who voted for Wright. First, let's see who else they voted for to get an idea what they their preferences are. We'll take a look at all the votes who listed Wright at the top and see who would have won the election had just those voters voted and Wright not been on the ballot.

In [9]:
new_ballots = []
for ballot in all_ballots:
    if ballot.ranked_candidates[0] == wright and len(ballot.ranked_candidates) > 1:
        new_ballots.append(Ballot(ballot.ranked_candidates[1:]))

In [10]:
election_result = instant_runoff_voting(candidates, new_ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Dan Smith          882  Active
Andy Montroll      849  Active
Bob Kiss           281  Rejected
James Simpson       78  Rejected
Write-in            19  Rejected
Kurt Wright          0  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Dan Smith         1002  Elected
Andy Montroll      984  Rejected
Bob Kiss             0  Rejected
James Simpson        0  Rejected
Write-in             0  Rejected
Kurt Wright          0  Rejected
Blank votes        123  Rejected



We can see that 882 people would have voted for Dan Smith, 849 for Andy Montroll, and only 281 for Bob Kiss. It's clear that Wright voters preferred Montroll, the Democrat, over Kiss, the Progressive.

So let's see what would have happened had some of those voters been strategic. Let's say they get together and all the voters who voted for Wright, then Montroll or Wright, then Smith, then Montroll, decide to switch Montroll with Wright. So now they list Montroll first and Wright either second or third (behind Smith).

In [11]:
new_ballots = []
for ballot in all_ballots:
    if ballot.ranked_candidates == (wright, montroll):
        new_ballots.append(Ballot([montroll, wright]))
    elif ballot.ranked_candidates == (wright, smith, montroll):
        new_ballots.append(Ballot([montroll, smith, wright]))
    else:
        new_ballots.append(ballot)

In [12]:
election_result = instant_runoff_voting(candidates, new_ballots)
winners = election_result.get_winners()
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Bob Kiss          2585  Active
Kurt Wright       2571  Active
Andy Montroll     2440  Active
Dan Smith         1306  Rejected
Write-in            36  Rejected
James Simpson       35  Rejected

ROUND 2
Candidate        Votes  Status
-------------  -------  --------
Bob Kiss          2981  Active
Andy Montroll     2931  Active
Kurt Wright       2914  Rejected
Dan Smith            0  Rejected
Write-in             0  Rejected
James Simpson        0  Rejected
Blank votes        147  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Andy Montroll     4063  Elected
Bob Kiss          3476  Rejected
Kurt Wright          0  Rejected
Dan Smith            0  Rejected
Write-in             0  Rejected
James Simpson        0  Rejected
Blank votes       1434  Rejected



Now we have Andy Montroll as the election winner. A preferred outcome for the Wright voters that they achieved by **not** voting their preferences and instead gaming the system.

## Ranked-Choice Voting Doesn't Always Result in the Best Head-to-head Candidate

Now let's look at whether the election results were the same as the head-to-head matchup. We don't need to change any ballots for this simulation, just count up the head-to-head preferences.

In [13]:
prefer_wright = 0
prefer_montroll = 0
for ballot in all_ballots:
    for candidate in ballot.ranked_candidates:
        if candidate == wright:
            prefer_wright += 1
            break
        elif candidate == montroll:
            prefer_montroll += 1
            break

In [14]:
print(f"In a heads-up competition {prefer_montroll} voters prefer Montroll and {prefer_wright} voters prefer Wright")

In a heads-up competition 4596 voters prefer Montroll and 3662 voters prefer Wright


In [15]:
prefer_kiss = 0
prefer_montroll = 0
for ballot in all_ballots:
    for candidate in ballot.ranked_candidates:
        if candidate == kiss:
            prefer_kiss += 1
            break
        elif candidate == montroll:
            prefer_montroll += 1
            break

In [16]:
print(f"In a heads-up competition {prefer_montroll} voters prefer Montroll and {prefer_kiss} voters prefer Kiss")

In a heads-up competition 4063 voters prefer Montroll and 3476 voters prefer Kiss


This show that more people wanted Andy Montroll than Bob Kiss, and, in addition, more people wanted Andry Montroll than Kurt Wright. And not by a little - these are substantial margins. But, unfortunately, ranked-choice voting thwarted the will of the voters in this case and gave them a mayor that fewer people wanted.

## Ranked-Choice Voting Allows for Election Spoilers

Was there a spoiler in the election? What would have happened if Wright wasn't in the race at all? Let's run that simulation.

In [17]:
new_ballots = []
for ballot in all_ballots:
    new_ballot = Ballot([c for c in ballot.ranked_candidates if c != wright])
    new_ballots.append(new_ballot)

In [18]:
election_result = instant_runoff_voting(candidates, new_ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Andy Montroll     2911  Active
Bob Kiss          2866  Active
Dan Smith         2188  Rejected
James Simpson      113  Rejected
Write-in            55  Rejected
Kurt Wright          0  Rejected
Blank votes        840  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Andy Montroll     4063  Elected
Bob Kiss          3476  Rejected
Dan Smith            0  Rejected
James Simpson        0  Rejected
Write-in             0  Rejected
Kurt Wright          0  Rejected
Blank votes       1434  Rejected



Montroll would have won. Thus Wright was a spoiler.

## Ranked-Choice Voting Allows for Candidates to Do Better but Receive Worse Outcomes

Let's look at the weird case, where doing better could have cause a candidate to perform worse in a ranked-choice voting election. This gets a little complex so to simplify things I'm going to remove candidates that aren't the top three. This happens anyway after the first round, so it doesn't affect the results at all, it just makes things easier to explain.

#### Top Three Only Ballots

In [19]:
three_candidate_ballots = []
for ballot in all_ballots:
    new_ballot = Ballot([c for c in ballot.ranked_candidates if c in {wright, kiss, montroll}])
    three_candidate_ballots.append(new_ballot)

We'll run the election again, juist to show the numbers between the three main candidates haven't changed.

In [20]:
election_result = instant_runoff_voting(candidates, three_candidate_ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Kurt Wright       3292  Active
Bob Kiss          2981  Active
Andy Montroll     2553  Rejected
Dan Smith            0  Rejected
James Simpson        0  Rejected
Write-in             0  Rejected
Blank votes        147  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Bob Kiss          4313  Elected
Kurt Wright       4058  Rejected
Andy Montroll        0  Rejected
Dan Smith            0  Rejected
James Simpson        0  Rejected
Write-in             0  Rejected
Blank votes        602  Rejected



OK. So let's say that Kiss campaigns extra hard and convinces more voters that he is the best candidate. Let's say that he convinces all the Wright > Kiss > Montroll voters to change their ballot to Kiss > Wright > Montroll. Then he convinces another 400 of the voters who only voted for Wright to vote for him instead. Let's see how this affects the results.

In [21]:
new_ballots = []
counter = 0
for ballot in three_candidate_ballots:
    if ballot.ranked_candidates == (wright, kiss, montroll):
        new_ballots.append(Ballot([kiss, wright, montroll]))
    elif ballot.ranked_candidates == (wright,) and counter < 400:
        counter += 1
        new_ballots.append(Ballot([kiss]))
    else:
        new_ballots.append(ballot)
        

In [22]:
election_result = instant_runoff_voting(candidates, new_ballots)
election_result.get_winners()
print(election_result)

ROUND 1
Candidate        Votes  Status
-------------  -------  --------
Bob Kiss          3723  Active
Andy Montroll     2553  Active
Kurt Wright       2550  Rejected
Write-in             0  Rejected
James Simpson        0  Rejected
Dan Smith            0  Rejected
Blank votes        147  Rejected

FINAL RESULT
Candidate        Votes  Status
-------------  -------  --------
Andy Montroll     4063  Elected
Bob Kiss          3876  Rejected
Kurt Wright          0  Rejected
Write-in             0  Rejected
James Simpson        0  Rejected
Dan Smith            0  Rejected
Blank votes       1034  Rejected



Suddenly, Bob Kiss get many more votes, starts out with a huge lead after the first round, and loses the election. Note that this didn't happen in the real election. I don't know how likely this would be in a real election. My guess is not very often because the numbers need to align pretty well. However, it's definitely possible.

## Alternatives to Ranked-choice Voting

In the next post, I'll talk about approval voting and why I believe it is a good alternative to ranked-choice voting.